In [1]:
!pip install -e ../../

Obtaining file:///C:/Users/netxph/Projects/mlpouch
  Attempting uninstall: mlpouch
    Found existing installation: mlpouch 0.0.1
    Uninstalling mlpouch-0.0.1:
      Successfully uninstalled mlpouch-0.0.1
  Running setup.py develop for mlpouch


In [2]:
import pandas as pd
from mlpouch.decomposition import FunkSVD
import logging

logging.basicConfig(level=logging.DEBUG)

In [3]:
movies = pd.read_csv("data/movies_clean.csv")
del movies['Unnamed: 0']
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [4]:
reviews = pd.read_csv("data/reviews_clean.csv")
del reviews['Unnamed: 0']
reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [5]:
reviews_sub = reviews[["user_id", "movie_id", "rating"]]
reviews_sub.head()

,user_id,movie_id,rating
0,1,68646,10
1,1,113277,10
2,2,422720,8
3,2,454876,8
4,2,790636,7


In [6]:
movie_reviews = reviews_sub.groupby(['user_id', 'movie_id'])['rating'].max().unstack()
movie_reviews.head()

movie_id,8,10,12,25,91,417,439,443,628,833,...,8144778,8144868,8206708,8289196,8324578,8335880,8342748,8342946,8402090,8439854
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
movie_reviews_sub = movie_reviews[[73486, 75314,  68646, 99685]].dropna(axis=0)
movie_reviews_sub

movie_id,73486,75314,68646,99685
user_id,,,,
265,10.0,10.0,10.0,10.0
1023,10.0,4.0,9.0,10.0
1683,8.0,9.0,10.0,5.0
6571,9.0,8.0,10.0,10.0
11639,10.0,5.0,9.0,9.0
13006,6.0,4.0,10.0,6.0
14076,9.0,8.0,10.0,9.0
14725,10.0,5.0,9.0,8.0
23548,7.0,8.0,10.0,8.0


In [8]:
dec = FunkSVD(iters=10, learning_rate=0.005)
movie_reviews_pred = dec.fit_transform(movie_reviews_sub)

DEBUG:root:Optimization Statistics
DEBUG:root:Iterations | Mean Squared Error 
DEBUG:root:1 		 44.347969980297364
DEBUG:root:2 		 15.431987497565032
DEBUG:root:3 		 4.210814767018258
DEBUG:root:4 		 2.83335573569919
DEBUG:root:5 		 2.708435108227014
DEBUG:root:6 		 2.6735312787181025
DEBUG:root:7 		 2.6479465710312664
DEBUG:root:8 		 2.6213815979613693
DEBUG:root:9 		 2.5914785185229987
DEBUG:root:10 		 2.5572842944276624


In [9]:
movie_reviews_pred

movie_id,73486,75314,68646,99685
user_id,,,,
265,10.135782,8.910022,10.492441,10.202056
1023,8.660390,7.662690,8.944108,8.769901
1683,7.964290,6.841064,8.276559,7.728831
6571,9.337931,8.422272,9.809452,9.818978
11639,8.725153,7.389278,8.937033,8.474029
13006,6.860919,5.901223,7.390149,6.737655
14076,9.347676,8.093872,9.794022,8.974842
14725,8.164831,7.240468,8.515069,8.278511
23548,8.425074,7.254970,9.101764,8.426084
